<a href="https://colab.research.google.com/github/nishntmish/Cnn-Keraas-Tuner/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 4.5 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [4]:
print(tf.__version__)

2.7.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [8]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [9]:
train_images=train_images/255.0
test__images=test_images/255.0

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 13m 22s]
val_accuracy: 0.9128333330154419

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 01h 11m 02s
INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        48432     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 96)                2230368   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2,282,682
Trainable params: 2,282,682
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 253s 150ms/step - loss: 0.1615 - accuracy: 0.9394 - val_loss: 0.2682 - val_accuracy: 0.9048
Epoch 5/10
1688/1688 [==============================] - 254s 150ms/step - loss: 0.1237 - accuracy: 0.9532 - val_loss: 0.2814 - val_accuracy: 0.9148
Epoch 6/10
1688/1688 [==============================] - 254s 150ms/step - loss: 0.0934 - accuracy: 0.9651 - val_loss: 0.2937 - val_accuracy: 0.9158
Epoch 7/10
1688/1688 [==============================] - 254s 150ms/step - loss: 0.0704 - accuracy: 0.9744 - val_loss: 0.3062 - val_accuracy: 0.9130
Epoch 8/10
1688/1688 [==============================] - 254s 150ms/step - loss: 0.0529 - accuracy: 0.9803 - val_loss: 0.3617 - val_accuracy: 0.9125
Epoch 9/10
1688/1688 [==============================] - 254s 150ms/step - loss: 0.0457 - accuracy: 0.9831 - val_loss: 0.3863 - val_accuracy: 0.9108
Epoch 10/10
1688/1688 [==============================] - 254s 150ms/step - loss: 0.0358 - accuracy: 0.9872 - val